# Lab 1

Group Members:

- Parker

- Suma 

- Chris

- Oliver

## Introduction

**Why is it important to find this kind of bias in machine learning models?**

TODO: Answer this

**Why will the type of investigation I am performing be relevant to other researchers or practitioners?**

TODO: Answer this

For example, we are trying to answer if a romantic comedy is ranked as more positive or if a horror movie is ranked as more negative. 

To do this, we utilize the dataset of IMDB reviews that are pre-labeled as positive or negative and contain movie genres.
We will analyze the bias using the word embeddings of GloVe & ConceptNet. Each of these embeddings will be applied to the movie reviews to determine the overall sentiment of the review. 

We believe that a "lesser" embedding will perform more poorly in the face of a conflicting sentiment lexicon. By this, we expect that "horror" movies may have more "negative" ratings because of the "negative" words used in the reviews to describe the content of the movie, versus the overall context of the review as positive or negative. If the embedding is more narrow-focused, like Glove, it may produce results that bias towards romantic comedies as more positive, since the overall content of the review should have more "positive" words based on the content of the movie. However, we expect an embedding that has a wider knowledge graph focus, like ConceptNet, to remove this bias and focus solely on the review content.

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn
import re

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

C:\Users\schack\AppData\Local\Temp\ipykernel_1372\2373303691.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#this code is taken from LectureNotesMaster/01 ConceptNet.ipynb notebook shared by Prof E Larson/Robyn Speer

def load_embeddings(filename):
    """
    Load a DataFrame from the generalized text format used by word2vec, GloVe,
    fastText, and ConceptNet Numberbatch. The main point where they differ is
    whether there is an initial line with the dimensions of the matrix.
    """
    labels = []
    rows = []
    with open(filename, encoding='utf-8') as infile:
        for i, line in enumerate(infile):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                # This is a header row giving the shape of the matrix
                continue
            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    
    arr = np.vstack(rows)
    return pd.DataFrame(arr, index=labels, dtype='f')

embeddings = load_embeddings('data/glove.840B.300d.txt')
embeddings.shape

(2196017, 300)

In [3]:
#Loading the lexicon
def load_lexicon(filename):
    """
    Load a file from Bing Liu's sentiment lexicon
    (https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), containing
    English words in Latin-1 encoding.
    
    One file contains a list of positive words, and the other contains
    a list of negative words. The files contain comment lines starting
    with ';' and blank lines, which should be skipped.
    """
    lexicon = []
    with open(filename, encoding='latin-1') as infile:
        for line in infile:
            line = line.rstrip()
            if line and not line.startswith(';'):
                lexicon.append(line)
    return lexicon

pos_words = load_lexicon('data/positive-words.txt')
neg_words = load_lexicon('data/negative-words.txt')

print(len(pos_words), len(neg_words))

2006 4783


In [4]:
#train the model
pos_words_common = list(set(pos_words) & set(embeddings.index)) 
neg_words_common = list(set(neg_words) & set(embeddings.index)) 

pos_vectors = embeddings.loc[pos_words_common]
neg_vectors = embeddings.loc[neg_words_common]
print(pos_vectors.shape,neg_vectors.shape)

(1974, 300) (4642, 300)


In [5]:
#train the inputs and outputs
vectors = pd.concat([pos_vectors, neg_vectors])
targets = np.array([1 for entry in pos_vectors.index] + [-1 for entry in neg_vectors.index])
labels = list(pos_vectors.index) + list(neg_vectors.index)

In [6]:
#prep for train test split
train_vectors, test_vectors, train_targets, test_targets, train_labels, test_labels = \
    train_test_split(vectors, targets, labels, test_size=0.1, random_state=0)

In [7]:
# create a linear classifier 
model = SGDClassifier(loss='log_loss', random_state=0, max_iter=100)
model.fit(train_vectors, train_targets)
accuracy_score(model.predict(test_vectors), test_targets)



0.9546827794561934

In [8]:


def vecs_to_sentiment(vecs):
    # predict_log_proba gives the log probability for each class
    predictions = model.predict_log_proba(vecs)

    # To see an overall positive vs. negative classification in one number,
    # we take the log probability of positive sentiment minus the log
    # probability of negative sentiment.
    # this is a logarithm of the max margin for the classifier, 
    # similar to odds ratio (but not exact) log(p_1/p_0) = log(p_1)-log(p_0)
    return predictions[:, 1] - predictions[:, 0]


def words_to_sentiment(words):
    vecs = embeddings.loc[words].dropna()
    log_odds = vecs_to_sentiment(vecs)
    return pd.DataFrame({'sentiment': log_odds}, index=vecs.index)


# Show 20 examples from the test set
words_to_sentiment(test_labels).iloc[:20]



,sentiment
annoying,-11.879341
courageousness,3.137851
clunky,-10.792788
resent,-10.085441
enchant,8.348746
nicer,6.943111
excites,5.942516
deceptively,-1.065492
limited,-1.134500
hinder,-2.021496


In [9]:
#tokenize using regular expressions

TOKEN_RE = re.compile(r"\w.*?\b")
# The regex above finds tokens that start with a word-like character (\w), and continues
# matching characters (.+?) until the next word break (\b). It's a relatively simple
# expression that manages to extract something very much like words from text.


def text_to_sentiment(text):
    # tokenize the input phrase
    tokens = [token.casefold() for token in TOKEN_RE.findall(text)]
    # send each token separately into the embedding, then the classifier
    sentiments = words_to_sentiment(tokens)
    return sentiments['sentiment'].mean() # return the mean for the classifier

In [10]:
#Load the movie list for Comedy
df_cat1_movies = pd.read_csv('data/movie_dataset/1_movies_per_genre/Comedy.csv', sep = ',')
df_cat1_movies.head

<bound method NDFrame.head of                                      name  year movie_rated run_length  \
0                 Guardians of the Galaxy  2014       PG-13    2h 1min   
1                      Back to the Future  1985          PG   1h 56min   
2                            Finding Nemo  2003           G   1h 40min   
3                                      Up  2009          PG   1h 36min   
4                         The Truman Show  1998          PG   1h 43min   
..                                    ...   ...         ...        ...   
95                               3 Idiots  2009       PG-13   2h 50min   
96                                Yes Man  2008       PG-13   1h 44min   
97                         Men in Black 3  2012       PG-13   1h 46min   
98                       Wedding Crashers  2005           R   1h 59min   
99  Anchorman: The Legend of Ron Burgundy  2004       PG-13   1h 34min   

               genres              release_date  rating  num_raters  \
0         

In [11]:
df_cat2_movies = pd.read_csv('data/movie_dataset/1_movies_per_genre/Horror.csv', sep = ',')
df_cat2_movies.head

<bound method NDFrame.head of                     name  year movie_rated run_length  \
0            The Shining  1980           R   2h 26min   
1                  Alien  1979           R   1h 57min   
2            World War Z  2013       PG-13   1h 56min   
3                 Psycho  1960           R   1h 49min   
4      Shaun of the Dead  2004           R   1h 39min   
..                   ...   ...         ...        ...   
95             From Hell  2001           R    2h 2min   
96         The Orphanage  2007           R   1h 45min   
97        Train to Busan  2016       TV-MA   1h 58min   
98          Evil Dead II  1987           R   1h 24min   
99  Underworld Awakening  2012           R   1h 28min   

                         genres             release_date  rating  num_raters  \
0               Drama; Horror;        13 June 1980 (USA)     8.4      860290   
1              Horror; Sci-Fi;        22 June 1979 (USA)     8.4      762793   
2   Action; Adventure; Horror;        21 June

In [12]:
#I feel name, year and genres cols in the csv files are sufficient to retain, drop the rest, add cols back in as needed
df_cat1_movies_lim=df_cat1_movies[['name', 'year',  'genres']]
df_cat2_movies_lim=df_cat2_movies[['name', 'year', 'genres']]

Interesting  problem - there is movie titled Evil Dead thats a Comedy;Horror mix. May be we need load Cat1 to some other genre?

For the movies in the dataframes we have loaded, bring in the reviews

In [26]:
df_concat_movies = pd.concat([df_cat1_movies_lim, df_cat2_movies_lim])
df_concat_movies['file_name'] = df_concat_movies['name'] + ' ' + df_concat_movies['year'].astype(str) + '.csv'

df_concat_movies.head()

,name,year,genres,file_name
0,Guardians of the Galaxy,2014,Comedy;,Guardians of the Galaxy 2014.csv
1,Back to the Future,1985,Comedy;,Back to the Future 1985.csv
2,Finding Nemo,2003,Comedy;,Finding Nemo 2003.csv
3,Up,2009,Comedy;,Up 2009.csv
4,The Truman Show,1998,Comedy;,The Truman Show 1998.csv


In [30]:
#https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
# and https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
import os
directory = 'data/movie_dataset/2_reviews_per_movie_raw'

dfs = list()

# iterate over files in
# that directory
for filename in os.listdir(directory):	
	print('looking for :', filename) 
	if df_concat_movies['file_name'].eq(filename).any():
		print("Found:", df_concat_movies['file_name'])
		data = pd.read_csv(os.path.join(directory, filename), header=None)
		data['filename'] = filename
		dfs.append(data)
df = pd.concat(dfs, ignore_index=True)
df.head()



looking for : 10 Cloverfield Lane 2016.csv
looking for : 10 Things I Hate About You 1999.csv
looking for : 12 Angry Men 1957.csv
looking for : 12 Monkeys 1995.csv
looking for : 12 Strong 2018.csv
looking for : 12 Years a Slave 2013.csv
looking for : 127 Hours 2010.csv
looking for : 13 Hours 2016.csv
looking for : 1408 2007.csv
looking for : 1917 2019.csv
looking for : 1941 1979.csv
looking for : 2001_ A Space Odyssey 1968.csv
looking for : 2012 2009.csv
looking for : 21 2008.csv
looking for : 21 Jump Street 2012.csv
looking for : 22 Jump Street 2014.csv
looking for : 28 Days Later... 2002.csv
looking for : 28 Weeks Later 2007.csv
looking for : 3 Idiots 2009.csv
looking for : 30 Days of Night 2007.csv
looking for : 300 2006.csv
looking for : 3_10 to Yuma 2007.csv
looking for : 42 2013.csv
looking for : 50 First Dates 2004.csv
looking for : 500 Days of Summer 2009.csv
looking for : 50_50 2011.csv
looking for : 54 1998.csv
looking for : 8 Mile 2002.csv
looking for : A Beautiful Mind 2001.

,0,1,2,3,4,5,6,filename
0,username,rating,helpful,total,date,title,review,10 Cloverfield Lane 2016.csv
1,Imme-van-Gorp,7,102,123,30 January 2019,Unfortunately the ending ruined an otherwise ...,This movie is full of suspense. It makes you g...,10 Cloverfield Lane 2016.csv
2,sonofocelot-1,5,385,500,10 May 2016,...oh dear Abrams. Again.\r\n,I'll leave this review fairly concise. <br/><b...,10 Cloverfield Lane 2016.csv
3,mhodaee,5,110,143,4 August 2017,"Fantastic, gripping, thoroughly enjoyable, un...",I give the 5/10 out of the credit I owe to the...,10 Cloverfield Lane 2016.csv
4,fil-nik09,5,73,100,5 October 2016,Hmmm...\r\n,"First of all, I must say that I was expecting ...",10 Cloverfield Lane 2016.csv


In [32]:
#TODO: We need to add the header in to the appended reviews file.

## Investigation

As part of your assignment, you will choose a methodology that involves comparing two (or more) techniques to one another. Discuss how you will measure a difference between the two techniques. That is, if you are measuring the difference statistically, what test will you use and why is it appropriate? Are there any limitations to performing this test that you should be aware of?

Source: https://ieee-dataport.org/open-access/imdb-movie-reviews-dataset

Dataset has 1 million reviews from 1150 movies spread across 17 genres; there's also other meta data such as the IMDb rating and movie rating. 

Reference Paper: https://ieeexplore-ieee-org.proxy.libraries.smu.edu/document/9276893 

### Embeddings

### Evaluation metric

F-score

TODO: Explain why

### Hypothesis

TODO: Write hypothesis

## Observations

Carryout your analysis and model training. Explain your steps in as much detail so that the instructor can understand your code. 

Present results from your analysis and provide evidence from the results that support or refute your hypothesis. Write a conclusion based upon the various analyses you performed. Be sure to reference your research questions systematically in your conclusion. With your analysis complete, are there any additional research questions or limitations to your conclusions?

### Loading the dataset

### Cleaning/preparing the dataset

### Model training


### Output

## Conclusion

TODO: Add conclusion

Identify two conferences or journals that would be interested in the results of your analysis.  

## References